# Pipeline Training - PCB Connector Recognition

Pipeline completa in due step:
1. **Classificatore OCCLUSION vs VISIBLE**
2. **Autoencoder per anomaly detection su OK visibili**

## 📚 Struttura Notebook

Questa pipeline è organizzata in notebook separati:
- **`step1_occlusion_classifier.ipynb`** - Training classificatore OCCLUSION
- **`step2_autoencoder.ipynb`** - Training autoencoder
- **`step3_inference.ipynb`** - Funzione di inferenza unica

**Utilizzo su Colab:**
1. Esegui questo notebook per la preparazione dataset
2. Esegui `step1_occlusion_classifier.ipynb` per STEP 1
3. Esegui `step2_autoencoder.ipynb` per STEP 2
4. Esegui `step3_inference.ipynb` per testare l'inferenza

Oppure esegui i notebook in sequenza usando `%run`.

## Setup e Dipendenze


In [ ]:
# Installa dipendenze se necessario
# !pip install torch torchvision pandas pillow numpy tqdm


In [ ]:
# Setup: Clona repository GitHub e monta Google Drive per i dati
import os
from pathlib import Path

# Opzione 1: Clona da GitHub (consigliato per sviluppo)
# Sostituisci con il tuo repository URL
GITHUB_REPO = "https://github.com/TUO_USERNAME/TUO_REPO.git"  # ⚠️ MODIFICA QUESTO!
REPO_DIR = "/content/project"

# Clona repository (se non esiste già)
if not Path(REPO_DIR).exists():
    !git clone {GITHUB_REPO} {REPO_DIR}
else:
    # Se esiste già, fai pull per aggiornare
    os.chdir(REPO_DIR)
    !git pull

# Cambia directory al repository
os.chdir(REPO_DIR)
print(f"Repository directory: {os.getcwd()}")

# Opzione 2: Monta Google Drive solo per i dati (immagini)
from google.colab import drive
drive.mount('/content/drive')

# Path ai dati su Drive
DATA_ROOT = Path("/content/drive/MyDrive/Project Work/Data")
print(f"Data directory: {DATA_ROOT}")

import torch
import torch.nn as nn
from pathlib import Path
import pandas as pd
import numpy as np

# Verifica GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"Memoria GPU disponibile: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


## STEP 0: Preparazione Dataset

Prepara il CSV con i path completi alle immagini.


In [ ]:
# Prepara dataset
# I path delle immagini devono puntare a Google Drive
import pandas as pd
from pathlib import Path

input_csv = Path("features_labeled.csv")
print(f"Leggendo CSV: {input_csv.absolute()}")
print(f"CSV esiste: {input_csv.exists()}")

df = pd.read_csv(input_csv)
print(f"\nCSV caricato: {len(df)} righe")
print(f"Colonne: {df.columns.tolist()}")

# Verifica colonne necessarie
required_cols = ['connector_name', 'filename', 'label']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print(f"⚠️  Colonne mancanti: {missing_cols}")
    print(f"Colonne disponibili: {df.columns.tolist()}")

df['label_merged'] = df['label'].replace('PARTIAL OCCLUSION', 'OCCLUSION')

# Path delle immagini su Google Drive
# Verifica prima se esiste già una colonna image_path
if 'image_path' in df.columns:
    print("\n⚠️  Trovata colonna 'image_path' esistente")
    print("Prime righe:")
    print(df[['image_path', 'connector_name', 'filename']].head())
    # Aggiorna i path per puntare a Drive
    df['image_path'] = df.apply(
        lambda row: f"/content/drive/MyDrive/Project Work/Data/connectors/{row['connector_name']}/{row['filename']}",
        axis=1
    )
else:
    # Crea path da zero
    df['image_path'] = df.apply(
        lambda row: f"/content/drive/MyDrive/Project Work/Data/connectors/{row['connector_name']}/{row['filename']}",
        axis=1
    )

print(f"\nVerificando esistenza immagini...")
print(f"Path Drive: /content/drive/MyDrive/Project Work/Data/connectors/")
print(f"Drive montato: {Path('/content/drive').exists()}")

# Verifica esistenza immagini
existing = []
for idx, path in enumerate(df['image_path']):
    if Path(path).exists():
        existing.append(idx)
    elif idx < 5:  # Stampa i primi 5 path non trovati per debug
        print(f"  ❌ Non trovato: {path}")

print(f"\nImmagini trovate: {len(existing)}/{len(df)}")

if len(existing) == 0:
    print("\n⚠️  PROBLEMA: Nessuna immagine trovata!")
    print("\nPossibili cause:")
    print("1. Drive non montato correttamente")
    print("2. Path Drive diverso da quello atteso")
    print("3. Struttura cartelle diversa su Drive")
    print("\nVerifica manualmente:")
    print("  !ls -la /content/drive/MyDrive/Project\\ Work/Data/connectors/")
    
    # Prova a trovare dove sono le immagini
    print("\nCercando immagini su Drive...")
    import subprocess
    result = subprocess.run(['find', '/content/drive/MyDrive', '-name', '*.png', '-type', 'f'], 
                          capture_output=True, text=True, timeout=10)
    if result.returncode == 0 and result.stdout:
        sample_paths = result.stdout.strip().split('\n')[:5]
        print(f"Trovate immagini in:")
        for p in sample_paths:
            print(f"  {p}")
            # Estrai il path base
            if 'connectors' in p:
                base_path = '/'.join(p.split('connectors')[0].split('/')[:-1]) + 'connectors'
                print(f"\n  Path base suggerito: {base_path}")
                break

df_valid = df.iloc[existing].copy() if existing else df.copy()
df_valid['label'] = df_valid['label_merged']
output_df = df_valid[['image_path', 'label', 'connector_name']].copy()

output_csv = Path("data/dataset.csv")
output_csv.parent.mkdir(parents=True, exist_ok=True)
output_df.to_csv(output_csv, index=False)

print(f"\n✅ Dataset preparato: {len(output_df)} righe")
if len(output_df) > 0:
    print(f"Distribuzione label:")
    print(output_df['label'].value_counts())
else:
    print("⚠️  Dataset vuoto! Controlla i path delle immagini.")


## STEP 1: Training Classificatore OCCLUSION vs VISIBLE

Classificatore binario per distinguere immagini occluse da immagini visibili.


In [ ]:
# STEP 1: Importa e definisci classi e funzioni
# Nota: Su Colab, puoi eseguire step1_occlusion_classifier.ipynb prima
# oppure importare da qui. Per semplicità, includiamo il codice necessario.

# Vedi step1_occlusion_classifier.ipynb per il codice completo
# Qui assumiamo che tu abbia già eseguito quel notebook o importi le funzioni

print("⚠️  Esegui prima step1_occlusion_classifier.ipynb oppure importa le funzioni")
print("    Per ora, usa: %run step1_occlusion_classifier.ipynb")


In [ ]:
# Training del classificatore
# Se hai eseguito step1_occlusion_classifier.ipynb, la funzione è già disponibile
# Altrimenti, esegui: %run step1_occlusion_classifier.ipynb

# model_occ = train_occlusion_classifier(
#     csv_path="data/dataset.csv",
#     val_fraction=0.2,
#     batch_size=32,
#     num_epochs=20,
#     learning_rate=0.001,
#     device=device
# )

print("Esegui step1_occlusion_classifier.ipynb per il training del classificatore")


## STEP 2: Training Autoencoder per Anomaly Detection

Autoencoder addestrato solo su immagini OK per rilevare anomalie.


In [ ]:
# STEP 2: Importa e definisci classi e funzioni
# Vedi step2_autoencoder.ipynb per il codice completo

print("⚠️  Esegui step2_autoencoder.ipynb per il training dell'autoencoder")


In [ ]:
# Training dell'autoencoder
# Se hai eseguito step2_autoencoder.ipynb, la funzione è già disponibile
# Altrimenti, esegui: %run step2_autoencoder.ipynb

# model_ae = train_autoencoder(
#     csv_path="data/dataset.csv",
#     batch_size=32,
#     num_epochs=30,
#     learning_rate=0.001,
#     device=device
# )

print("Esegui step2_autoencoder.ipynb per il training dell'autoencoder")


In [ ]:
# Calcolo threshold
# Viene eseguito automaticamente in step2_autoencoder.ipynb

print("Il threshold viene calcolato automaticamente in step2_autoencoder.ipynb")


## STEP 3: Test Inferenza

Testa la funzione di inferenza unica su alcune immagini.


In [ ]:
# STEP 3: Funzione di inferenza
# Vedi step3_inference.ipynb per il codice completo

print("⚠️  Esegui step3_inference.ipynb per testare l'inferenza")


In [ ]:
# Test inferenza
# Esegui step3_inference.ipynb per testare la classificazione

print("Esegui step3_inference.ipynb per testare l'inferenza su immagini di esempio")


## Riepilogo

✅ **Modelli salvati in:**
- `models/occlusion_cnn.pth` - Classificatore OCCLUSION vs VISIBLE
- `models/ae_conv.pth` - Autoencoder
- `models/ae_threshold.npy` - Threshold per anomaly detection

✅ **Funzione di inferenza:**
```python
from step3_inference import classify_connector
result = classify_connector("path/to/image.png")
# Ritorna: "OK", "KO" o "OCCLUSION"
```

### Download Modelli

Dopo il training, scarica i modelli per usarli localmente:
```python
from google.colab import files
files.download('models/occlusion_cnn.pth')
files.download('models/ae_conv.pth')
files.download('models/ae_threshold.npy')
```
